# Article Personalization Trained Using Preferences

This notebook focuses on testing the main hypothesis of the project that human preferences from pairs can be used to train an LLM model to better predict the article the human is most likely to prefer to read next. 

# Implementation

In [1]:
## Creating Corpus

In [1]:
links = []
with open('./data/small-dataset.txt') as f:
    links = f.read().splitlines() 

In [2]:
links[0]

'https://www.greaterwrong.com/posts/Y9Yqux7iPwpvnppyS/engineering-experience-through-score'

In [3]:
from dataclasses import dataclass

@dataclass
class Content:
    content_id: int
    url: str
    content_type: str
    content_text: str

In [4]:
from bs4 import BeautifulSoup
from urllib.request import urlopen, Request

def bs_fetcher(url):
    text = None
    user_agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
    try:
        req = Request(url)
        req.add_header('User-Agent', user_agent)
        page = urlopen(req)
        html = page.read().decode("utf-8")
        soup = BeautifulSoup(html, "html.parser")
        text = soup.get_text()
         
    except Exception as e:
        print('failed to fetch ', url)
    return text

In [5]:
# Testing bs_fetcher function
bs_fetcher('https://www.edge.org/response-detail/26557')

"\n\n\n\n\n\n\n\n\n\nEdge.org\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nSkip to main content\n\n\nCopyright © 2023 By Edge Foundation, Inc. All Rights Reserved.\n\n\n\n \n\nEdge.org\n\n\n\n\n\n\n\n \n\n\n\nTo arrive at the edge of the world's knowledge, seek out the most complex and sophisticated minds, put them in a room together, and have them ask each other the questions they are asking themselves.\n\n\n\n\nhttps://www.edge.org/response-detail/26557Printed On Sun November 19th 2023 \n\n\n\nSun, Nov 19, 2023HOMECONVERSATIONSVIDEOAUDIOANNUAL QUESTIONEVENTSNEWSLIBRARYABOUTPEOPLE \n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n2016 : WHAT DO YOU CONSIDER THE MOST INTERESTING RECENT [SCIENTIFIC] NEWS? WHAT MAKES IT IMPORTANT?\n\n\n\n In the News [ 22 ] \n \xa0\xa0|\xa0\xa0 \n Contributors [ 199 ] \n \xa0\xa0|\xa0\xa0 \n View All Responses [ 199 ]  \n\n\n\n\n  \n Pamela McCorduck \n Author, Machines Who Think, The Universal Machine, Bounded Rationality, This Could Be Important; Co-author (with Edward 

In [6]:
def fetch_content(links) -> list[Content]:
    return [Content(content_id=i, url=url, content_type='article', content_text=bs_fetcher(url)) 
            for i, url in enumerate(links)]

In [7]:
import pandas as pd

contents_df = pd.DataFrame([o.__dict__ for o in fetch_content(links) if o.content_text])

failed to fetch  https://www.crazypi.com/nvidia-jetson-nano-2gb
failed to fetch  http://lukemuehlhauser.com/about/


In [8]:
contents_df


,content_id,url,content_type,content_text
0,0,https://www.greaterwrong.com/posts/Y9Yqux7iPwp...,article,\n\nEngineering Experience Through Score - Les...
1,1,https://www.greaterwrong.com/posts/YMtZRGLbvdD...,article,\n\nGeneralized Efficient Markets in Political...
2,2,https://www.greaterwrong.com/posts/ZJzSxo6nCNv...,article,\n\nWhy Planning is Hard: A Multifaceted Model...
3,3,https://www.greaterwrong.com/posts/znfkdCoHMAN...,article,\n\nThe ground of optimization - LessWrong 2.0...
4,4,https://www.greaterwrong.com/s/oFePMp9rKftEeZDDr,article,\n\nLawful Truth - LessWrong 2.0 viewerArchive...
5,5,https://www.greaterwrong.com/s/oi873FWi6pHWxswSa,article,\n\nThe Science of Winning at Life - LessWrong...
6,6,https://www.coursera.org/specializations/whart...,article,Business Strategies for A Better World Special...
7,8,https://www.cs.cmu.edu/~02251/schedule.html,article,\n\n\n\n\n\n02-251: Great Ideas in Computation...
8,9,https://www.cs.cmu.edu/~15251/course-info.html,article,\n\n\n\n\n15-251 Fall 2018\n\n\n\n\n\n\n15-251...
9,10,https://slatestarcodex.com/2013/02/17/90-of-al...,article,\n\n\n\n90% of all claims about the problems w...


In [9]:
import pickle

# Saving loaded data as a binary file
pickle.dump(contents_df, open( "./temp/fetched-content.data", "wb" ))

## Creating Pipeline For Collecting Preferences From User

In [10]:
import random, itertools

MAX_PAIRS = 10
order = list(range(len(contents_df)))

pairs = list(itertools.combinations(order, 2))
random.shuffle(pairs)


prefs = []

for x, y in pairs[:MAX_PAIRS]:
    cont_x = contents_df.iloc[x]
    cont_y = contents_df.iloc[y]
    print(cont_x.url , '\n', cont_y.url)
    
    pref = input('Type t for top and b for bottom link : ')
    c, r = None, None
    if pref == 't':
        c, r = cont_x, cont_y
    elif pref == 'b':
        c, r = cont_y, cont_x
    else:
        print('Invalid input, skipping....')
    
    print('--------------')
    prefs.append({'chosen': c.content_text, 'rejected': r.content_text})

https://www.edge.org/response-detail/26557 
 https://theory.cs.northwestern.edu/courses/
--------------
http://www.nwlink.com/~donclark/about/about.html 
 https://teachyourselfcs.com/
--------------
https://www.greaterwrong.com/posts/znfkdCoHMANwqc2WE/the-ground-of-optimization-1 
 https://theory.cs.northwestern.edu/courses/
--------------
https://www.greaterwrong.com/posts/Y9Yqux7iPwpvnppyS/engineering-experience-through-score 
 https://www.greaterwrong.com/s/oFePMp9rKftEeZDDr
--------------
https://slatestarcodex.com/2014/07/30/meditations-on-moloch/ 
 http://www.math.toronto.edu/ilia/Teaching/MAT337.2018/index.html
--------------
https://www.greaterwrong.com/posts/ZJzSxo6nCNvod67Xs/why-planning-is-hard-a-multifaceted-model 
 https://teachcomputerscience.com/synchronous-and-asynchronous/
--------------
http://math.huji.ac.il/~mhochman/research-expo.html 
 http://stellar.mit.edu/S/course/18/fa08/18.415/materials.html
--------------
https://www.coursera.org/specializations/wharton-glob

In [37]:
pref_df = pd.DataFrame(prefs)

In [40]:
pref_df = pd.DataFrame(prefs)
pref_df

,chosen,rejected
0,\n\n\n\n\n\n\n\n\n\nEdge.org\n\n\n\n\n\n\n\n\n...,\n\n\n\n\nTeaching – Northwestern CS Theory Gr...
1,\n\nTeach Yourself Computer Science\n\n\n\n\n\...,\n\n\nAbout page for Donald Clark\n\n\n\n\n\n\...
2,\n\nThe ground of optimization - LessWrong 2.0...,\n\n\n\n\nTeaching – Northwestern CS Theory Gr...
3,\n\nLawful Truth - LessWrong 2.0 viewerArchive...,\n\nEngineering Experience Through Score - Les...
4,\n\n\n\nMAT337. Introduction to Real Analysis\...,\n\n\n\nMeditations On Moloch | Slate Star Cod...
5,\n\nWhy Planning is Hard: A Multifaceted Model...,Synchronous and Asynchronous Data Transmissi...
6,\n\n\n\nStellar : Message of the Day\n\n\n\n\n...,\n\nDynamical systems theory\n\n\n Dynamical S...
7,Business Strategies for A Better World Special...,\n\nAI Safety Fundamentals Course\n\n\n\n\n\n\...
8,\n\n18.408 - Fall '16\n\n\n18.408 Topics in Th...,\n\n\n\n\n\n02-251: Great Ideas in Computation...
9,\n\nThe Science of Winning at Life - LessWrong...,\n\nGeneralized Efficient Markets in Political...


In [25]:
pref_df.to_csv('./pref_data.csv')


In [2]:
pref_df = pd.read_csv('./pref_data.csv')

In [8]:
# s = pref_df[['chosen', 'rejected']].to_json('./pref_data.json')

In [13]:
s = pref_df[['chosen', 'rejected']].to_json('./pref_data.json', orient='records')

In [14]:
len(pref_df)

10

In [29]:
ld_pref = load_dataset('csv', data_files='./pref_data.csv', split='train')

In [41]:
ld_pref['chosen']

['\n\nWhy Planning is Hard: A Multifaceted Model\n - LessWrong 2.0 viewerArchiveSequencesAboutSearchLog InQuestionsEventsShortformAlignment ForumAF CommentsHomeFeaturedAllTagsRecent CommentsWhy Planning is Hard: A Multifaceted Model\nRuby31 Mar 2019 2:33 UTC29 points9 commentsLW linkPlanning & Decision-Making\uf141Post permalinkLink without commentsLink without top nav barsLink without comments or top nav barsContentsContentsWhat is plan\xadning? Plan\xadning is a Pre\xaddic\xadtion/\u200bIn\xadfor\xadma\xadtion ProblemPlan\xadning is a Com\xadpu\xadta\xadtion ProblemPlan\xadning is a Self-Knowl\xadedge & Self-Mastery ProblemPre\xaddict\xading yourselfKnow\xading what you wantSelf-mas\xadtery of your hu\xadman brainHeuris\xadtics and biasesUs\xading Sys\xadtem 1 (in\xadtu\xadition) and Sys\xadtem 2 (ex\xadplicit rea\xadson) in harmonyEmo\xadtional MasteryPlan\xadning is a Re\xadcur\xadsive ProblemSum\xadmary: What It Takes to Be a Great PlannerEndnotesEpistemic con\xadfi\xaddence: High

In [31]:
df

Dataset({
    features: ['chosen', 'rejected'],
    num_rows: 160800
})

### Load trained model



In [2]:
from transformers import OPTForCausalLM, AutoTokenizer, OPTForSequenceClassification, AutoModelForSeq2SeqLM, AutoModelForSequenceClassification, AutoModel

/Users/harish/opt/anaconda3/envs/personal-content-ranker/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
# https://towardsdatascience.com/linear-regression-with-hugging-face-3883fe729324

model = AutoModelForSequenceClassification.from_pretrained('output/checkpoint-5', num_labels = 1)
tokenizer = AutoTokenizer.from_pretrained('output/checkpoint-5')

Some weights of T5ForSequenceClassification were not initialized from the model checkpoint at google/flan-t5-small and are newly initialized: ['classification_head.dense.bias', 'classification_head.dense.weight', 'classification_head.out_proj.bias', 'classification_head.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [30]:
import pandas as pd

# Testing with stored preferences
pref_df = pd.read_csv('./pref_data.csv')

for index, row in pref_df.iterrows():
    # Extract chosen and text input
    chosen_article_text_input = tokenizer(pref_df.loc[index, 'chosen'], return_tensors="pt", truncation=True)
    rejected_article_text_input = tokenizer(pref_df.loc[index, 'rejected'], return_tensors="pt", truncation=True)

    # Using model with chosen article and rejected article
    chosen_article_output = model(**chosen_article_text_input)
    rejected_article_output = model(**rejected_article_text_input)

    chosen_article_score = chosen_article_output.logits.numpy()[0, 0]
    rejected_article_score = rejected_article_output.logits.numpy()[0, 0]

    pref_df.loc[index, 'chosen_article_score'] = chosen_article_score
    pref_df.loc[index, 'rejected_article_score'] = rejected_article_score
    if chosen_article_score > rejected_article_score: 
        pref_df.loc[index, 'model_results'] = "correct" 
    else: 
        pref_df.loc[index, 'model_results'] = "wrong"

pref_df

,Unnamed: 0,chosen,rejected,chosen_article_score,rejected_article_score,model_results
0,0,\n\nWhy Planning is Hard: A Multifaceted Model...,\n\n\n\n\n\n\nA research workflow with Zotero ...,0.068711,-0.083770,correct
1,1,Synchronous and Asynchronous Data Transmissi...,\n\nGeneralized Efficient Markets in Political...,-0.034582,0.185842,wrong
2,2,\n\n\n\nMAT337. Introduction to Real Analysis\...,\n\n\n\nStellar : Message of the Day\n\n\n\n\n...,0.251424,0.121554,correct
3,3,\n\n\n\n\n\n\nA research workflow with Zotero ...,\n\n\n\n\n\n\n\n\n\nEdge.org\n\n\n\n\n\n\n\n\n...,-0.083770,-0.170541,correct
4,4,\n\n\n\n90% of all claims about the problems w...,\n\n\n\n\nOff the Convex Path – Off the convex...,-0.232459,0.064567,wrong
5,5,\n\nAI Safety Fundamentals Course\n\n\n\n\n\n\...,\n\n\n\nThe Control Group Is Out Of Control | ...,0.018573,-0.109004,correct
6,6,\n\n\n\nMAT337. Introduction to Real Analysis\...,\n\n\n\n\n\n02-251: Great Ideas in Computation...,0.251424,0.095170,correct
7,7,\n\n\n\nStellar : Message of the Day\n\n\n\n\n...,\n\n\n\n\n15-251 Fall 2018\n\n\n\n\n\n\n15-251...,0.121554,-0.112996,correct
8,8,\n\nThe ground of optimization - LessWrong 2.0...,\n\n\n\n\n\n\nA research workflow with Zotero ...,0.116302,-0.083770,correct
9,9,\n\n\n\n\n\n\n\n\n\nRent Seeking - Econlib\n\n...,\n\n\n\n\nOff the Convex Path – Off the convex...,-0.014000,0.064567,wrong
